This anlaysis was conducted in a Google Colaboratory (Colab) session which allows users to use a virtual GPU. This notebook is presented here as a demo of how to use the code in this repository.

## Setup
Colab creates a virtual Linux machine, so Linux/Unix terminal commands can run in cells to manage folders and files. The cell below creates a test_images folder, downloads the model weights, and installs the SAM Github repository.

In [1]:
#Setup code - should only need to be run once
#Before running real code, be sure to also enable a CUDA GPU:
  #Runtime --> change runtime type --> T4 GPU (free) or other GPU of choice
#This will hold the manually uploaded image we'll work with in this notebook
!mkdir test_images
!mkdir output_images
!mkdir sam_weights
# Download model checkpoint into /content/sam_weights
# Use wget to download from the url provided on the github and then install into /content/ subfolder using -P to specify folder path
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth -P /content/sam_weights
# Install Segment Anything by cloning the git repository
!pip install git+https://github.com/facebookresearch/segment-anything.git

--2024-07-25 23:45:32--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.254.15, 108.157.254.124, 108.157.254.102, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.254.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘/content/sam_weights/sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   319MB/s    in 7.2s    

2024-07-25 23:45:39 (342 MB/s) - ‘/content/sam_weights/sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-xjvpoz0j
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-xjvpoz0j
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e2795658

This procedure makes use of geospatial libraries like rasterio and geopandas to manipulate the mask output for easy vectorizing and, eventually, measurements. If running in Colab, you'll need to manually install rasterio. Otherwise, if running this notebook in a regular Jupyter notebook session, you will need to make sure you have this installed in the Python environment you're using.


In [2]:
#Not included with Colab in stock version
#Need to install once per session
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 91.4 MB/s eta 0:00:00


The advantage of using Colab is that many common data science libraries come preinstalled. Again, if you run this locally in a Jupyter notebook you'll have to make sure the libraries below are installed.

In [3]:
#Import necessary libraries

#Import SAM functions
#This needs to be cloned and installed from Github (see above)
#e.g. pip install git+https://github.com/facebookresearch/segment-anything.git
from segment_anything import SamPredictor, sam_model_registry, SamAutomaticMaskGenerator
#Import other necessary libraries
import os
#Open and convert images
import cv2
#Display images and plots
import matplotlib.pyplot as plt
#Work with masks (which are created as numpy arrays)
import numpy as np
#For the purposes of this analysis, we're treating the masks as geographic features
#This allows us to make area and shape calculations easily
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
import rasterio
import rasterio.features
#Calculations
import math

Import the functions from the repository to use in analysis

In [5]:
from SAM_source_functions import *

## Run

In [6]:
#Define checkpoint and model type for use later
checkpoint = os.path.join('/content/', "sam_weights", "sam_vit_h_4b8939.pth")
#I think this could also just be 'default'
model_type = 'vit_h'

There are two functions in the repository: ```calculate_metrics()``` which saves a dataframe of metrics, and ```calculate_metrics_with_images()``` which saves a dataframe *and* saves images of each input with objects circles.

In [7]:
#Function outputs a dataframe as well as a list of masks and vectorized shapes (Shapely geometries)
#The masklist and geomlist can also be manually accessed later to do additional calculations
df, masklist, geomlist = calculate_metrics_with_images('/content/test_images', model_type, checkpoint, '/content/output_images')

In [8]:
#Save dataframe
df.to_excel('/content/SAM_output.xlsx')

In [9]:
#NOTE: to bulk download images from Colab, zip them first using this command (output .zip file destination/name followed by folder to zip)
# !zip -r /content/output_images.zip /content/output_images